In [1]:
import os, glob
import sys
import json
from PIL import Image
from collections import Counter

import numpy as np
import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
import tifffile as tiff
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import cv2

import pandas as pd

from sklearn.model_selection import KFold

sys.path.append("/kaggle/input/detection-wheel")

In [2]:
import torch
torch.__version__  #torch 2.0
#!nvidia-smi   #CUDA Version: 11.4
! ls /usr/local

bin   cuda-11.7  games	  lib  project_templates  share  waved
cuda  etc	 include  man  sbin		  src	 www


In [3]:
# # # Install pycocotools package
# import os
# !mkdir /kaggle/working/packages
# !cp -r /kaggle/input/pycocotools/* /kaggle/working/packages
# os.chdir("/kaggle/working/packages/pycocotools-2.0.6/")
# !python setup.py install -q
# !pip install . --no-index --find-links /kaggle/working/packages/ -q
# # # Install mmcv and mmdet packages
# # #3.0
# # #!pip install mmcv mmdet --no-index --find-links /kaggle/input/mmdetection/ -q
# # #os.chdir("/kaggle/working")
# # #ytt 2140
# # !pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
# # !pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
# # !pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
# # !pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
# # !pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
# # !pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
# # !pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

# # !rm -rf mmdetection

# # !cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
# # !mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
# # %cd /kaggle/working/mmdetection
# # !pip install -e .


# # 必要なライブラリのインストール（オフライン用）
# !pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/addict-2.4.0-py3-none-any.whl
# !pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/yapf-0.32.0-py2.py3-none-any.whl
# !pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/terminal-0.4.0-py3-none-any.whl
# !pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/terminaltables-3.1.10-py2.py3-none-any.whl
# #ytt
# #!pip install /kaggle/input/2023-hhp-mmdet/mmcv_full-1.7.0-cp310-cp310-manylinux1_x86_64_cu113.whl
# !pip install /kaggle/input/mmdet3-wheels/mmcv_full-1.7.1-cp310-cp310-linux_x86_64.whl
# #!pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/mmcv_full-1.7.0-cp37-cp37m-linux_x86_64.whl
# #!pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl
# #!pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl

# !cp -r /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/mmdetection/ /kaggle/working/
# %cd /kaggle/working/mmdetection
# !pip install -e . --no-deps
# %cd /kaggle/working/

# !pip install /kaggle/input/mmdetection-2-26-0/mmdetection-2-26-0/mmdet-2.26.0-py3-none-any.whl



In [4]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib

def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str

In [5]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, imgs, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = imgs
        self.name_indices = [os.path.splitext(os.path.basename(i))[0] for i in imgs]

    def __getitem__(self, idx):
        # load images and masks
        img_path = self.imgs[idx]
        name = self.name_indices[idx]
        array = tiff.imread(img_path)
        img = Image.fromarray(array)
        
        img, _ = self.transforms(img, img)

        return img, name

    def __len__(self):
        return len(self.imgs)

In [6]:
# import torchvision
# import torchvision
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# def get_model_instance_segmentation(num_classes):
#     # load an instance segmentation model pre-trained on COCO
#     model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=None, weights_backbone=None)

#     # get number of input features for the classifier
#     in_features = model.roi_heads.box_predictor.cls_score.in_features
#     # replace the pre-trained head with a new one
#     model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#     # now get the number of input features for the mask classifier
#     in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
#     hidden_layer = 256
#     # and replace the mask predictor with a new one
#     model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                        hidden_layer,
#                                                        num_classes)

#     return model

In [7]:
# !pip install mmdet==2.26.0

In [8]:
!cd ../

In [9]:
import sys
sys.path.append('ex')

In [10]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    return T.Compose(transforms)

In [11]:
from engine import train_one_epoch, evaluate
import utils

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [13]:
import sys
import importlib
# importlib.reload(mmdet)
# sys.path.append('/home/nischay/hubmap/try_mm/mmdetection/')
# !sudo pip uninstall -y mmdet 
# sys.path.insert(0, '/home/nischay/hubmap/try_mm/mmdetection/cbnet')
sys.path.insert(9,'/home/nischay/hubmap/vitadap/detection')
# !cd /home/nischay/hubmap/try_mm/mmdetection/cbnet && sudo pip install -v -e .
# !pip install -e .
import mmdet
from mmdet.apis import init_detector, inference_detector,show_result_pyplot, set_random_seed
from mmdet.models import build_detector
#print(mmdet.__version__)
#print(mmcv.__version__)
#print(mmengine.__version__)

from mmdet.models.backbones import *
# # #check file her

from mmcv import Config

from mmdet.models.backbones.swin import SwinTransformer
from mmdet.models.backbones import cbnet
print(sys.path)


/usr/local/lib/python3.8/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


['/home/nischay/hubmap', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/root/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/local/lib/python3.8/dist-packages/ocotillo-1.0.5.1-py3.8.egg', '/usr/local/lib/python3.8/dist-packages/audio2numpy-0.1.2-py3.8.egg', '/home/nischay/hubmap/vitadap/detection', '/usr/lib/python3/dist-packages', '/kaggle/input/detection-wheel', 'ex']


In [14]:
# import mmdet, mmcv, mmengine
#from mmengine.config import Config
#from mmengine.runner import Runner
#from mmdet.utils import register_all_modules
#from mmdet.apis import init_detector, inference_detector
#from mmengine.visualization import Visualizer

from mmdet.apis import init_detector, inference_detector,show_result_pyplot, set_random_seed

#print(mmdet.__version__)
#print(mmcv.__version__)
#print(mmengine.__version__)


# # #check file her

from mmcv import Config

config_file = '/home/nischay/hubmap/vitadap/detection/work_dirs/hubmap/pretwsiallhtc_resnext101_exp3_augv4_maskloss4/detec101next.py'
cfg = Config.fromfile(config_file )



cfg.data.test.pipeline[1].img_scale= [(2048, 2048)]#train 1280-1600 , inf 1280-4000 -> 1440,2520,3600#

#test cfg
#cfg.model.test_cfg.rcnn.max_per_img = 1000
#cfg.model.test_cfg.rcnn.nms.iou_threshold=0.3
#cfg.model.test_cfg.rcnn.mask_thr_binary=0.45

cfg.seed = 0
set_random_seed(0, deterministic=False)
#print(f'Config:\n{cfg.pretty_text}')

In [15]:
print(f'Config:\n{cfg.data.test.pipeline}')

Config:
[{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': [(2048, 2048)], 'flip': True, 'flip_direction': ['horizontal', 'vertical'], 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]


In [16]:
FOLD = 1

In [17]:
# import json
import pandas as pd
import glob

In [18]:
df = pd.read_csv('coco/folds5.csv')

In [19]:
val_df = df[df['fold']==1]['id'].tolist()

In [20]:
tiles = pd.read_csv('/home/nischay/hubmap/Data/tile_meta.csv')

In [21]:
tiles = tiles[tiles['dataset']==3][tiles['source_wsi']<8].reset_index(drop=True)

/tmp/ipykernel_2916174/142878037.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tiles = tiles[tiles['dataset']==3][tiles['source_wsi']<8].reset_index(drop=True)


In [22]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from pathlib import Path
annFile = f"/home/nischay/hubmap/coco/ds1_coco_1024_valid_all_fold1.json"

# colors = ['Set1'] 
# legend = ids_categories #{1: 'blood_vessel'}



In [23]:
coco_gt = COCO(annFile)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [24]:
val_ids = tiles['id'].tolist()
val_ids = ['/home/nischay/hubmap/Data/train/'+i+'.tif' for i in val_ids]


In [25]:
# checkpoint_file = '/home/nischay/hubmap/try_mm/mmdetection/work_dirs/hubmap/pretexp1_ds1ft_20e_2048_htc101/best_segm_mAP_epoch_14.pth'
# checkpoint_file = '/home/nischay/hubmap/try_mm/mmdetection/work_dirs/hubmap/pretexp10_ds1ft_20e_2048_htc101_pretdsall/best_segm_mAP_epoch_19.pth'
checkpoint_file = '/home/nischay/hubmap/vitadap/detection/work_dirs/hubmap/pretwsiallhtc_resnext101_exp3_augv4_maskloss4/best_segm_mAP_epoch_17.pth'
#model = init_detector(config_file, checkpoint_file, device=device)  # or device='cuda:0'
model = init_detector(cfg, checkpoint_file, device=device)  # or device='cuda:0'
# model = get_model_instance_segmentation(num_classes=2)
# model.to(device)
# model.load_state_dict(torch.load('/kaggle/input/hubmap-train/fold_0_epoch5.pth'))
# model.eval()
# print()

load checkpoint from local path: /home/nischay/hubmap/vitadap/detection/work_dirs/hubmap/pretwsiallhtc_resnext101_exp3_augv4_maskloss4/best_segm_mAP_epoch_17.pth


In [26]:
all_imgs = val_ids #glob.glob('/kaggle/input/hubmap-hacking-the-human-vasculature/test/*.tif')
dataset_test = PennFudanDataset(all_imgs, get_transform(train=False))
test_dl = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=os.cpu_count(), pin_memory=True)

In [27]:
# all_imgs

In [28]:
len(all_imgs)

1200

In [29]:
# (pred[0][0]).shape

In [30]:
sample = None
import mmcv

ids = []
heights = []
widths = []
prediction_strings = []
confidence_thresholds = {0: 0.5, 1: 0.5, 2: 0.8}

for img in tqdm(all_imgs):
    img_array = mmcv.imread(img,channel_order='rgb')
    [h, w, c] = img_array.shape 
    pred = inference_detector(model,img)
    pred_string = ''
    #print(pred.pred_instances)
    #print (len(pred)) #2
    #print(pred)
    
    pred_class = pred[0]
#     print(len(pred_class)) #3 class
#     print(pred_class)
#     print(pred_class[0].shape) #(5,5) #class 0 with 5 item
#     print(pred_class[1].shape) #(40,5)#class 1 with 40 item
#     print(pred_class[2].shape) #(1,5) #class 2 with 1 item
    #print(pred.pred_instances)
    
    pred_mask = pred[1]

                        
    ids.append(os.path.basename(img).split('.')[0])
    heights.append(h)
    widths.append(w)
    prediction_strings.append(pred)                    
    
    # prediction_strings.append(pred_string)                    


  0%|                                                  | 0/1200 [00:00<?, ?it/s]/home/nischay/hubmap/vitadap/detection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
/home/nischay/hubmap/vitadap/detection/mmdet/models/roi_heads/mask_heads/fcn_mask_head.py:245: UserWarning: Scale_factor should be a Tensor or ndarray with shape (4,), float would be deprecated. 
  warn('Scale_factor should be a Tensor or ndarray '
100%|███████████████████████████████████████| 1200/1200 [53:09<00:00,  2.66s/it]


In [31]:
config_file2 = '/home/nischay/hubmap/try_mm/mmdetection/cbnet/work_dirs/hubmap/pseudo_exp1_withpret_cbbase_1600_morepretep/exp1_cbnet_swinb.py'

cfg2 = Config.fromfile(config_file2)

cfg2.data.test.pipeline[1].img_scale= [(1600, 1600)]#train 1280-1600 , inf 1280-4000 -> 1440,2520,3600#
cfg2.model.test_cfg.rcnn.score_thr = 0.001

cfg2.model.test_cfg.rcnn.max_per_img = 100
cfg2.model.test_cfg.rcnn.nms.type='nms'
cfg2.model.test_cfg.rcnn.nms.iou_threshold=0.5

cfg2.model.test_cfg.rcnn.mask_thr_binary=0.5

cfg2.seed = 0
set_random_seed(0, deterministic=False)
#print(f'Config:\n{cfg.pretty_text}')


# checkpoint_file2 = '/home/nischay/hubmap/try_mm/mmdetection/work_dirs/hubmap/pseudo/pseudo60combpretexp5_ds1ft_20e_2048_htc50/best_segm_mAP_epoch_15.pth'
# checkpoint_file2 = '/home/nischay/hubmap/try_mm/mmdetection/work_dirs/hubmap/pseudo/pseudo60combpretexp5_ds1ft_20e_2048_htc50_augv3/detectors_epoch_23.pth'
checkpoint_file2 = '/home/nischay/hubmap/try_mm/mmdetection/cbnet/work_dirs/hubmap/pseudo_exp1_withpret_cbbase_1600_morepretep/best_segm_mAP_epoch_15.pth'
model2 = init_detector(cfg2, checkpoint_file2, device=device)  # or device='cuda:0'



sample = None
import mmcv

ids2 = []
heights2 = []
widths2 = []
prediction_strings2 = []

for img in tqdm(all_imgs):
    img_array = mmcv.imread(img,channel_order='rgb')
    [h, w, c] = img_array.shape 
    pred = inference_detector(model2,img)
    pred_string = ''
    #print(pred.pred_instances)
    #print (len(pred)) #2
    #print(pred)
    
    pred_class = pred[0]
#     print(len(pred_class)) #3 class
#     print(pred_class)
#     print(pred_class[0].shape) #(5,5) #class 0 with 5 item
#     print(pred_class[1].shape) #(40,5)#class 1 with 40 item
#     print(pred_class[2].shape) #(1,5) #class 2 with 1 item
    #print(pred.pred_instances)
    
    pred_mask = pred[1]

                        
    ids2.append(os.path.basename(img).split('.')[0])
    heights2.append(h)
    widths2.append(w)
    prediction_strings2.append(pred)                    
    
    # prediction_strings.append(pred_string)                    


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load checkpoint from local path: /home/nischay/hubmap/try_mm/mmdetection/cbnet/work_dirs/hubmap/pseudo_exp1_withpret_cbbase_1600_morepretep/best_segm_mAP_epoch_15.pth


100%|███████████████████████████████████████| 1200/1200 [27:12<00:00,  1.36s/it]


In [32]:
import mmcv_custom  # noqa: F401,F403
import mmdet_custom  # noqa: F401,F403

In [33]:

config_file3 = '/home/nischay/hubmap/vitadap/detection/hubconf/exp4_adapbeitv2l.py'

cfg3 = Config.fromfile(config_file3)
cfg3.data.test.pipeline

cfg3.data.test.pipeline[1].img_scale= [(1400, 1400)]#train 1280-1600 , inf 1280-4000 -> 1440,2520,3600#
cfg3.model.test_cfg.rcnn.score_thr = 0.001

cfg3.model.test_cfg.rcnn.max_per_img = 100
cfg3.model.test_cfg.rcnn.nms.type='nms'

cfg3.model.test_cfg.rcnn.nms.iou_threshold=0.5
cfg3.model.test_cfg.rcnn.mask_thr_binary=0.5

# cfg3.data.test.pipeline[1].img_scale= [(1600, 1600)]#train 1280-1600 , inf 1280-4000 -> 1440,2520,3600#

cfg3.seed = 0
set_random_seed(0, deterministic=False)
#print(f'Config:\n{cfg.pretty_text}')


# checkpoint_file2 = '/home/nischay/hubmap/try_mm/mmdetection/work_dirs/hubmap/pseudo/pseudo60combpretexp5_ds1ft_20e_2048_htc50/best_segm_mAP_epoch_15.pth'
# checkpoint_file3 = '/home/nischay/hubmap/vitadap/detection/work_dirs/pretexp4_adaplargehtc_1200_bs2_augv4/best_segm_mAP_epoch_21.pth'\

checkpoint_file3 = '/home/nischay/hubmap/vitadap/detection/work_dirs/ds12combine/beitv2ladap_exp2/f1/best_segm_mAP_epoch_18.pth'
# checkpoint_file3 = '/home/nischay/hubmap/try_mm/mmdetection/cbnet/work_dirs/hubmap/exp2_withpret1_cbbase_1600_morepretep//kaggle/input/pretexp5-adapv2lbeithtc-cv428f1/detectors_epoch_23.pth.pth'
model3 = init_detector(cfg3, checkpoint_file3, device='cuda:0')  # or device='cuda:0'


sample = None
import mmcv

ids3 = []
heights3 = []
widths3 = []
prediction_strings3 = []

for img in tqdm(all_imgs):
    img_array = mmcv.imread(img,channel_order='rgb')
    [h, w, c] = img_array.shape 
    pred = inference_detector(model3,img)
    pred_string = ''
    #print(pred.pred_instances)
    #print (len(pred)) #2
    #print(pred)
    
    pred_class = pred[0]
#     print(len(pred_class)) #3 class
#     print(pred_class)
#     print(pred_class[0].shape) #(5,5) #class 0 with 5 item
#     print(pred_class[1].shape) #(40,5)#class 1 with 40 item
#     print(pred_class[2].shape) #(1,5) #class 2 with 1 item
    #print(pred.pred_instances)
    
    pred_mask = pred[1]

                        
    ids3.append(os.path.basename(img).split('.')[0])
    heights3.append(h)
    widths3.append(w)
    prediction_strings3.append(pred)                    
    
    # prediction_strings.append(pred_string)                    




load checkpoint from local path: /home/nischay/hubmap/vitadap/detection/work_dirs/ds12combine/beitv2ladap_exp2/f1/best_segm_mAP_epoch_18.pth


100%|███████████████████████████████████████| 1200/1200 [46:44<00:00,  2.34s/it]


In [34]:
from skimage import filters
from scipy import ndimage as ndi
from scipy.ndimage import binary_fill_holes
from skimage.segmentation import find_boundaries
from skimage.morphology import binary_dilation, binary_erosion

def smooth_boundary(mask_pred):
    # Smooth boundaries by dilating and eroding
    # smoothed_mask = binary_dilation(mask_pred)
    # smoothed_mask = binary_erosion(smoothed_mask)

    # Remove boundaries from the mask
    boundary_mask = find_boundaries(mask_pred)
    mask_pred = mask_pred & ~boundary_mask
    
    return mask_pred

import numpy as np

import numpy as np
from scipy.ndimage import convolve

def weighted_post_processing(mask_pred):
    kernel = np.array([[0.05, 0.4, 0.4],
                       [0.05, 0.95, 0.4],
                       [0.05, 0.4, 0.4]])
    weights = convolve(mask_pred.astype(float), kernel, mode='constant')

    # Apply weights to the mask
    weighted_mask_pred = mask_pred * weights

    return weighted_mask_pred
# Example usage
# mask_pred_weighted = weighted_post_processing(mask_pred)



In [35]:
import copy
import pycocotools.coco as coco
from pycocotools.cocoeval import COCOeval
from pycocotools import mask
from skimage.morphology import binary_dilation
from skimage.morphology import binary_erosion, binary_dilation, binary_opening, binary_closing
# prediction_strings2 = copy.deepcopy(ps2_copy)

# Convert predictions to COCO format
coco_preds2 = []

from skimage.measure import label, regionprops



In [36]:
MIN_PIXELS = 40
def nms_predictions(classes, scores, bboxes, masks, 
                    iou_th=.5, shape=(512, 512), weights=[0.5,0.5]):
    he, wd = shape[0], shape[1]
    boxes_list = [[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he]
                  for x in bboxes]
    scores_list = [x for x in scores]
    labels_list = [x for x in classes]
    nms_bboxes, nms_scores, nms_classes = nms(
        boxes=[boxes_list], 
        scores=[scores_list], 
        labels=[labels_list], 
        weights=weights,

        iou_thr=iou_th
    )
                        
    # nms_bboxes, nms_scores, nms_classes = weighted_boxes_fusion(
    #     boxes_list=[boxes_list], 
    #     scores_list=[scores_list], 
    #     labels_list=[labels_list], 
    #     weights=weights,
    #     conf_type='max',
    #     iou_thr=iou_th
    # )
                        
    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
        
        # nms_masks.append(masks[scores.index(s)])
    nms_scores, nms_classes, nms_masks = zip(*sorted(zip(nms_scores, nms_classes, nms_masks), reverse=True))
    return nms_classes, nms_scores, nms_masks

def ensemble_pred_masks(masks, min_pixels=MIN_PIXELS, shape=(512, 512)):
    result = []
    used = np.zeros(shape, dtype=int) 

    prev_masks = []
    new_masks = []
    for i, mask in enumerate(masks):
        # cont, hier = cv2.findContours(np.array(mask,dtype=np.uint8),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # if len(cont)>0:
            # for cnt in cont:
            #     convex_mask = cv2.fillConvexPoly(np.zeros_like(np.array(mask,dtype=np.uint8)),points=cnt, color=1)
            #     fillornot = len(pd.Series((convex_mask==mask).flatten()).value_counts())
            #     if fillornot>1: #fill
            #         mask = convex_mask

        
            # before= mask.sum()
        mask = binary_erosion(binary_dilation(mask))  #post processing 
            # if before!=mask.sum():
               # print('after',mask.sum(),'before',before)
        mask = mask * (1-used)
        # if mask.sum() >= 60: # skip predictions with small area
                    # used += mask 
                #     result.append(rle_encode(mask))
        new_masks.append(mask)
        
    return new_masks

In [37]:
MODELS = [prediction_strings, prediction_strings2]


In [38]:
for i in prediction_strings[0][0]:
    print(i.shape[0])

100


In [39]:
print(prediction_strings[0][0][0].shape[0])

100


In [63]:
len(prediction_strings2)

1200

In [41]:
from ex.wbf_tracking import *

In [42]:
# !pip install ensemble_boxes

In [43]:
from ensemble_boxes import *
MODEL_WEIGHTS = [0.25,0.75]
def bbox_to_key(bbox):
    return str(np.round(bbox, 6))


# Fuse masks that belong to fused boxes
def get_wsf_mask(wbf_box, wbf_org, pmasks, pmasks_lkup, thres=0.5):
    w, h = 512, 512
    mask = np.zeros((w, h), dtype=np.uint8)
    for i in range(len(wbf_org)):
        key = bbox_to_key(wbf_org[i][4:])
        model = int(wbf_org[i][3])
        # try:
        ind = pmasks_lkup[model][key]
        mask = mask + pmasks[model][ind]
        # except:
            # pass
    # convert thres to integer based on number of boxes
    threshold = max(1, int(thres*len(wbf_org)))
    # threshold = 0.0001
            
    # remove pixels outside WBF box
    m2 = np.zeros((w, h), dtype=np.uint8)
    x1 = max(0, int(h * wbf_box[0]))
    y1 = max(0, int(w * wbf_box[1]))
    x2 = min(h, int(h * wbf_box[2]))
    y2 = min(w, int(w * wbf_box[3]))
    # print(x1,x2,y1,y2)
    m2[y1:y2, x1:x2] = 1
    mask = (mask >= threshold) * m2
    return mask.astype(np.uint8)

In [44]:
def get_glomeruli_masks_single_image(image_id, data):
    masks = []
    for item in data:
        if item["id"] == image_id:
            anns = item["annotations"]
            for an in anns:
                category_type = an["type"]
                if category_type == "glomerulus":
                    segmentation = an["coordinates"]
                    mask_img = coordinates_to_masks(segmentation, (512, 512))[0]
                    masks.append(mask_img.astype(np.uint8))

    return masks

def coordinates_to_masks(coordinates, shape):
    masks = []
    for coord in coordinates:
        mask = np.zeros(shape, dtype=np.uint8)
        cv2.fillPoly(mask, [np.array(coord)], 1)
        masks.append(mask)
    return masks


In [45]:
jsonl_file_path = "Data/polygons.jsonl"
data = []
with open(jsonl_file_path, "r") as file:
    for line in file:
        data.append(json.loads(line))
        

In [46]:
os.path.basename(val_ids[0]).split('.')[0]


'000e79e206b7'

In [47]:
categories_list=['blood_vessel']
#------------------------------------------------------------------------------
categories_ids = {name:id for id, name in enumerate(categories_list)}  
ids_categories = {id:name for id, name in enumerate(categories_list)}  
categories =[{'id':id,'name':name} for name,id in categories_ids.items()]

print(categories_ids)
print(ids_categories)
print(categories)

{'blood_vessel': 0}
{0: 'blood_vessel'}
[{'id': 0, 'name': 'blood_vessel'}]


In [48]:

import json, cv2, numpy as np, itertools, random, pandas as pd
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from skimage import io
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm
from sklearn import model_selection

import matplotlib.pyplot as plt
from skimage import io
from pycocotools.coco import COCO
import matplotlib.patches as mpatches


def coordinates_to_masks(coordinates, shape):
    masks = []
    for coord in coordinates:
        mask = np.zeros(shape, dtype=np.uint8)
        cv2.fillPoly(mask, [np.array(coord)], 1)
        masks.append(mask)
    return masks

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(itertools.groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

def rle_to_binary_mask(mask_rle, shape=(1024, 1024)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) 
                       for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def coco_structure(images_ids):
    idx=1
    annotations=[]
    images=[]
    for item in tqdm(data,total=int(len(images_ids))):
        image_id=item["id"]
        if image_id in images_ids:
            image = {"id": image_id, "file_name": image_id + ".tif", "height": 512, "width": 512}
            images.append(image)
        else:continue
        #-----------------------------
        anns=item["annotations"]
        for an in anns:
            category_type=an["type"]
            if category_type =="blood_vessel":
                category_id=categories_ids[category_type]
                segmentation=an["coordinates"]
                mask_img = coordinates_to_masks(segmentation, (512, 512))[0]
                ys, xs = np.where(mask_img)
                x1, x2 = min(xs), max(xs)
                y1, y2 = min(ys), max(ys)

                rle = binary_mask_to_rle(mask_img)

                seg = {
                    "id": idx,
                    "image_id": image_id,
                    "category_id": categories_ids["blood_vessel"],
                    "segmentation": rle,
                    "bbox": [int(x1), int(y1), int(x2 - x1 + 1), int(y2 - y1 + 1)],
                    "area": int(np.sum(mask_img)),
                    "iscrowd": 0,
                }
                if image_id in images_ids:
                    annotations.append(seg)
                    idx=idx+1
                
    return {"info": {}, "licenses": [], "categories": categories, "images": images, "annotations": annotations}

In [49]:
# prediction_strings[0]

In [50]:
val_ids[0].split('/')[-1].split('.')[0]

'000e79e206b7'

In [66]:
subm_ids, subm_masks, subm_boxes = [], [], []
from skimage import measure
from ex.wbf_tracking import *


idx = 1
annotations = []
images = []
for id in tqdm(range(len(prediction_strings))):


    image_id = val_ids[id].split('/')[-1].split('.')[0]

    image = {"id": image_id, "file_name": image_id + ".tif", "height": 512, "width": 512}
    images.append(image)

    masks_nms_list = []
    pred_dict_list=  []
    score_nms_list = []
    box_nms_list = []
    class_nms_list = []
    
    
    for pred,weight in (zip(MODELS,MODEL_WEIGHTS)):
        pred_dict = {}
        previous_masks = []
        classes_nms = []
        scoresb_nms = []
        bboxesb_nms = []
        weightsb_nms = []        
        result = pred[id]
        
        #find the max class 
        c = []
        for i, classe in enumerate(result[0]):
            #print(classe.shape)
            c.append(classe.shape[0])
        
        maxclass = np.argwhere(np.array(c)==np.max(c))[0][0]
        #print(c,test_name,classe.shape[0],np.max(c))
        for i, classe in enumerate(result[0]):
            if maxclass==maxclass: #classe.shape != (0, 5):
                bbs = classe
                sgs = result[1][i]
                # print(sgs.shape)
                count = 0
                for bb, sg in zip(bbs,sgs):
                    box = bb[:4]
                    cnf = bb[4]
                    box = [box[0] / 512, box[1] / 512, box[2] / 512, box[3] / 512]
                    # box = [box[0] / 2048, box[1] / 2048, box[2] / 2048, box[3] / 2048]
                    
                    # print(box)
                    # box 
                    if cnf >= 0.001:
                        mask = np.array(sg,dtype=np.uint8)  
                        previous_masks.append(mask)
                        scoresb_nms.extend([cnf])
                        bboxesb_nms.extend([box])
                        # weights = [weight] * len()
                        weightsb_nms.extend([weight])
        # print(len(bboxesb_nms))
        # print(len(scoresb_nms))
                        pred_dict[bbox_to_key(box)] = count
                        count+= 1
                        classes_nms = [0] * len(previous_masks)
                
            masks_nms_list.append(np.array(previous_masks, dtype=np.uint8))
            score_nms_list.append(np.array(scoresb_nms))
            box_nms_list.append(np.array(bboxesb_nms))

            class_nms_list.append(np.array(classes_nms))
            pred_dict_list.append(pred_dict)

        
    
    # print(len(masks_nms))
    # print(len(classes_nms))

    # print(len(scoresb_nms))

    wbf_boxes, wbf_scores, _, wbf_originals = weighted_boxes_fusion_tracking(box_nms_list, 
                                                                             score_nms_list, 
                                                                             labels_list=class_nms_list, 
                                                                             iou_thr=0.6, 
                                                                             skip_box_thr=0.01)
    # classes_nms, scores_nms, masks_nms = nms_predictions(classes_nms, scoresb_nms, bboxesb_nms, masks_nms,iou_th=0.6,weights=[0.5]) 
    # masks_nms = ensemble_pred_masks(masks_nms, classes_nms) 
    
    fin_masks = []
    used = np.zeros((512,512), dtype=int)
    # print(len(wbf_boxes))
    for i in range(len(wbf_boxes)):
        
        mask = get_wsf_mask(wbf_boxes[i], wbf_originals[i], masks_nms_list, pred_dict_list, thres=0.2)
        
        # try:
        #     props = measure.regionprops(mask)
        # except:
        #     continue
        # # # if there are multiple separated masks, pick the larger one
        # areas = []
        # for a in range(len(props)):
        #     areas.append(props[a].area)
        # try:
        #     target = np.argmax(areas)
        # except:
        #     continue
        # # extract properties of interest 
        # major_axis_len = props[target].major_axis_length
        # check against limits
        # if major_axis_len >= major_axis_len_min:
        # mask = mask * (1-used)
        # check if mask is chopped up by previous detections
        # if len(measure.find_contours(mask, 0.5, positive_orientation='low')) == 1:
            # used += mask
        fin_masks.append(mask)
            
            # res.append(rle_encode(mask))
        # else:
            # print('{}: Chopped\'n\'dropped #{}')
        # else:
            # if DEBUG:
                # print('{}: Failed limits #{}'.format(fn.split('/')[-1], i))

        

    # print(len(fin_masks))
    fin_masks = ensemble_pred_masks(fin_masks) 

    for i in range(len(fin_masks)):
        mask_img = fin_masks[i]
        conf = wbf_scores[i]
        if conf>0.5:
            ys, xs = np.where(mask_img)
            x1, x2 = min(xs), max(xs)
            y1, y2 = min(ys), max(ys)
    
            rle = binary_mask_to_rle(mask_img)
    
            seg = {
                "id": idx,
                "image_id": image_id,
                "category_id": categories_ids["blood_vessel"],
                "segmentation": rle,
                "bbox": [int(x1), int(y1), int(x2 - x1 + 1), int(y2 - y1 + 1)],
                "area": int(np.sum(mask_img)),
                "iscrowd": 0,
            }
            annotations.append(seg)
    
    
            idx += 1

    if id % 100 == 0:
        final_dict = {"info": {}, "licenses": [], "categories": categories, "images": images, "annotations": annotations}
        output_file_path = f"pseudo/exp2_ds3wsi8_coco_ensthr_50.json"
        with open(output_file_path, "w", encoding="utf-8") as output_file:
            json.dump(final_dict, output_file, ensure_ascii=True, indent=4)
    
    
    # subm_masks.append(fin_masks)
    # subm_ids.append(wbf_scores)
    # subm_boxes.append(wbf_boxes)


    
        # for enc_mask in encoded_masks:
        #     subm_ids.append(test_name[:test_name.find('.')])
        #     subm_masks.append(enc_mask)

  0%|          | 0/1200 [00:00<?, ?it/s]

In [77]:
import gc
gc.collect()

0

In [76]:
del model,model2,model3

In [67]:
import json

def combine_coco_datasets(file1_path, file2_path):
    # Load the JSON files
    with open(file1_path, 'r') as file1:
        dataset1 = json.load(file1)
    
    with open(file2_path, 'r') as file2:
        dataset2 = json.load(file2)
    
    # Combine images and annotations
    combined_images = dataset1['images'] + dataset2['images']
    combined_annotations = dataset1['annotations'] + dataset2['annotations']
    
    # Update image and annotation IDs
    num_images1 = len(dataset1['images'])
    num_annotations1 = len(dataset1['annotations'])
    num_annotations2 = len(dataset2['annotations'])
    
    # for image in combined_images:
        # image['id'] += num_images1
    for i in range(0, num_annotations1 + num_annotations2):
        combined_annotations[i]['id'] = i
    # for annotation in combined_annotations:
        # annotation['id'] += num_annotations1
        # annotation['image_id'] += num_images1
    
    # Create the combined dataset
    combined_dataset = {
        'info': {},
        'licenses': [],
        'categories': dataset1['categories'],  # Assuming categories are the same in both datasets
        'images': combined_images,
        'annotations': combined_annotations
    }
    
    return combined_dataset


In [68]:
combine_pseudo = combine_coco_datasets('/home/nischay/hubmap/coco/ds2wsiall_coco_1024_train_fold1.json', '/home/nischay/hubmap/pseudo/exp2_ds3wsi8_coco_ensthr_50.json')




In [69]:
# Save the combined dataset to a JSON file
with open('pseudo/50_combined_ds2wsiall_exp2pseudo.json', 'w') as output_file:
    json.dump(combine_pseudo, output_file)

In [70]:
# (subm_masks[0])
len(combine_pseudo['images'])

2312

In [71]:
len(combine_pseudo['annotations'])

21147

In [72]:
# np.array(masks_nms)
len(final_dict['images'])

1200

In [73]:
len(final_dict['annotations'])

9389

In [59]:
# len(pred_ens_str[0][1][1])

In [60]:
# len(pred2[1][0]+pred2[1][0])

In [61]:
# len(pred_ens_str[0][1][1])

In [62]:
# print(prediction_strings)